In [28]:
import qsharp

In [29]:
#from qsharp-widgets import SpaceChart, EstimateDetails
# This refuses to work for some reason 

We are using 8 different configurations, 4 for 12 qubits and 4 for 24 qubits:
* 12 Qubits - Default Setup
* 12 Qubits - 1 Distillation Factory
* 12 Qubits - Trapped Ion Qubits
* 12 Qubits - Majorana Qubits

For 24 qubits:
* 24 Qubits - Default Setup (Baseline with default qubit type and default distillation factories)
* 24 Qubits - 1 Distillation Factory
* 24 Qubits - Trapped Ion Qubits
* 24 Qubits - Majorana Qubits

The main comparisons will be between the same number of qubits and different configs, but baseline 12 and 24 will be compared as well to see how the resources scale

In [50]:
%%qsharp
//Grover's Search taken from https://learn.microsoft.com/en-us/azure/quantum/tutorial-qdk-grovers-search?tabs=tabid-vscode#the-grovers-algorithm
// This is for 12 qubits

import Microsoft.Quantum.Convert.*;
import Microsoft.Quantum.Math.*;
import Microsoft.Quantum.Arrays.*;
import Microsoft.Quantum.Measurement.*;
import Microsoft.Quantum.Diagnostics.*;
import Microsoft.Quantum.ResourceEstimation.*;


@EntryPoint()
operation Main() : Result[] {
    let nQubits = 12;
    let iterations = CalculateOptimalIterations(nQubits);
    Message($"Number of iterations: {iterations}");

    // Use Grover's algorithm to find a particular marked state.
    let results = GroverSearch(nQubits, iterations, ReflectAboutMarked);
    return results;
}

operation GroverSearch(
    nQubits : Int,
    iterations : Int,
    phaseOracle : Qubit[] => Unit) : Result[] {

    use qubits = Qubit[nQubits];

    PrepareUniform(qubits);

    for _ in 1..iterations {
        // if (BeginEstimateCaching("GroverAESExample.EstimateIteration", SingleVariant())) {
            phaseOracle(qubits);
            ReflectAboutUniform(qubits);
            //EstimatingAES(qubits);
        //     EndEstimateCaching();
        // }
    }

    // Measure and return the answer.
    return MResetEachZ(qubits);
}

function CalculateOptimalIterations(nQubits : Int) : Int {
    if nQubits > 63 {
        fail "This sample supports at most 63 qubits.";
    }
    let nItems = 1 <<< nQubits; // 2^nQubits
    let angle = ArcSin(1. / Sqrt(IntAsDouble(nItems)));
    let iterations = Round(0.25 * PI() / angle - 0.5);
    return iterations;
}

operation ReflectAboutMarked(inputQubits : Qubit[]) : Unit {
    Message("Reflecting about marked state...");
    use outputQubit = Qubit();
    within {
        // We initialize the outputQubit to (|0⟩ - |1⟩) / √2, so that
        // toggling it results in a (-1) phase.
        X(outputQubit);
        H(outputQubit);
        // Flip the outputQubit for marked states.
        // Here, we get the state with alternating 0s and 1s by using the X
        // operation on every other qubit.
        for q in inputQubits[...2...] {
            X(q);
        }
        
    } apply {
        Controlled X(inputQubits, outputQubit);
        EstimatingAES(inputQubits);
    }
}

operation PrepareUniform(inputQubits : Qubit[]) : Unit is Adj + Ctl {
    for q in inputQubits {
        H(q);
    }
}

operation ReflectAboutAllOnes(inputQubits : Qubit[]) : Unit {
    Controlled Z(Most(inputQubits), Tail(inputQubits));
}

operation EstimatingAES(inputQubits : Qubit[]) : Unit {
    //Scaled values from scaling.py
    //Based on the values from Jang et al.
    AccountForEstimates(
        [TCount(65518), AuxQubitCount(4590), RotationCount(13094), RotationDepth(9538), MeasurementCount(12)],
        PSSPCLayout(), inputQubits);
}
operation ReflectAboutUniform(inputQubits : Qubit[]) : Unit {
    within {
        // Transform the uniform superposition to all-zero.
        Adjoint PrepareUniform(inputQubits);
        // Transform the all-zero state to all-ones
        for q in inputQubits {
            X(q);
        }
    } apply {
        // Now that we've transformed the uniform superposition to the
        // all-ones state, reflect about the all-ones state, then let the
        // within/apply block transform us back.
        ReflectAboutAllOnes(inputQubits);
    }


}

In [51]:
result_12_default = qsharp.estimate("Main()")
result_12_default['jobParams']
# 12 qubits, automatic number of distillation factories, and qubit_gate_ns_e3

{'qecScheme': {'name': 'surface_code',
  'errorCorrectionThreshold': 0.01,
  'crossingPrefactor': 0.03,
  'logicalCycleTime': '(4 * twoQubitGateTime + 2 * oneQubitMeasurementTime) * codeDistance',
  'physicalQubitsPerLogicalQubit': '2 * codeDistance * codeDistance',
  'maxCodeDistance': 50},
 'errorBudget': 0.001,
 'qubitParams': {'instructionSet': 'GateBased',
  'name': 'qubit_gate_ns_e3',
  'oneQubitMeasurementTime': '100 ns',
  'oneQubitGateTime': '50 ns',
  'twoQubitGateTime': '50 ns',
  'tGateTime': '50 ns',
  'oneQubitMeasurementErrorRate': 0.001,
  'oneQubitGateErrorRate': 0.001,
  'twoQubitGateErrorRate': 0.001,
  'tGateErrorRate': 0.001,
  'idleErrorRate': 0.001},
 'constraints': {'maxDistillationRounds': 3},
 'estimateType': 'singlePoint'}

In [53]:
result_12_default = qsharp.estimate("Main()")
result_12_default['jobParams']
# 12 qubits, automatic number of distillation factories, and qubit_gate_ns_e3

{'qecScheme': {'name': 'surface_code',
  'errorCorrectionThreshold': 0.01,
  'crossingPrefactor': 0.03,
  'logicalCycleTime': '(4 * twoQubitGateTime + 2 * oneQubitMeasurementTime) * codeDistance',
  'physicalQubitsPerLogicalQubit': '2 * codeDistance * codeDistance',
  'maxCodeDistance': 50},
 'errorBudget': 0.001,
 'qubitParams': {'instructionSet': 'GateBased',
  'name': 'qubit_gate_ns_e3',
  'oneQubitMeasurementTime': '100 ns',
  'oneQubitGateTime': '50 ns',
  'twoQubitGateTime': '50 ns',
  'tGateTime': '50 ns',
  'oneQubitMeasurementErrorRate': 0.001,
  'oneQubitGateErrorRate': 0.001,
  'twoQubitGateErrorRate': 0.001,
  'tGateErrorRate': 0.001,
  'idleErrorRate': 0.001},
 'constraints': {'maxDistillationRounds': 3},
 'estimateType': 'singlePoint'}

In [54]:
result_12_default.diagram.space

In [33]:
result_12_default.diagram.time

In [34]:
results_12_tfac_one= qsharp.estimate("Main()", 
params={'constraints': {'maxTFactories': 1}})
results_12_tfac_one
# 12 qubits, 1 distillation factory limited, and qubit_gate_ns_e3


Runtime,38 mins,"Total runtime This is a runtime estimate for the execution time of the algorithm. In general, the execution time corresponds to the duration of one logical cycle (11,600 nanosecs) multiplied by the 14,427,112 logical cycles to run the algorithm. If however the duration of a single T factory (here: 128,800 nanosecs) is larger than the algorithm runtime, we extend the number of logical cycles artificially in order to exceed the runtime of a single T factory."
rQOPS,810.26M,"Reliable quantum operations per second The value is computed as the number of logical qubits after layout (9,399) (with a logical error rate of 3.00e-16) multiplied by the clock frequency (86,206.90), which is the number of logical cycles per second."
Physical qubits,15.84M,"Number of physical qubits This value represents the total number of physical qubits, which is the sum of 15,809,118 physical qubits to implement the algorithm logic, and 33,320 physical qubits to execute the T factories that are responsible to produce the T states that are consumed by the algorithm."
Logical algorithmic qubits,9.40k,"Number of logical qubits for the algorithm after layout Laying out the logical qubits in the presence of nearest-neighbor constraints requires additional logical qubits. In particular, to layout the $Q_{\rm alg} = 4,603$ logical qubits in the input algorithm, we require in total $2 \cdot Q_{\rm alg} + \lceil \sqrt{8 \cdot Q_{\rm alg}}\rceil + 1 = 9,399$ logical qubits."
Algorithmic depth,14.43M,"Number of logical cycles for the algorithm To execute the algorithm using _Parallel Synthesis Sequential Pauli Computation_ (PSSPC), operations are scheduled in terms of multi-qubit Pauli measurements, for which assume an execution time of one logical cycle. Based on the input algorithm, we require one multi-qubit measurement for the 1,562 single-qubit measurements, the 654,700 arbitrary single-qubit rotations, and the 3,275,900 T gates, three multi-qubit measurements for each of the 1,050 CCZ and 0 CCiX gates in the input program, as well as 22 multi-qubit measurements for each of the 476,900 non-Clifford layers in which there is at least one single-qubit rotation with an arbitrary angle rotation."
Logical depth,196.35M,"Number of logical cycles performed This number is usually equal to the logical depth of the algorithm, which is 14,427,112. However, in the case in which a single T factory is slower than the execution time of the algorithm, we adjust the logical cycle depth to exceed the T factory's execution time."
Clock frequency,86.21k,Number of logical cycles per second This is the number of logical cycles that can be performed within one second. The logical cycle time is 12 microsecs.
Number of T states,17.68M,"Number of T states consumed by the algorithm To execute the algorithm, we require one T state for each of the 3,275,900 T gates, four T states for each of the 1,050 CCZ and 0 CCiX gates, as well as 22 for each of the 654,700 single-qubit rotation gates with arbitrary angle rotation."
Number of T factories,1,"Number of T factories capable of producing the demanded 17,683,500 T states during the algorithm's runtime The total number of T factories 1 that are executed in parallel is computed as $\left\lceil\dfrac{\text{T states}\cdot\text{T factory duration}}{\text{T states per T factory}\cdot\text{algorithm runtime}}\right\rceil = \left\lceil\dfrac{17,683,500 \cdot 128,800\;\text{ns}}{1 \cdot 2,277,634,804,800\;\text{ns}}\right\rceil$"
Number of T factory invocations,17.68M,"Number of times all T factories are invoked In order to prepare the 17,683,500 T states, the 1 copies of the T factory are repeatedly invoked 17,683,500 times."
Physical algorithmic qubits,15.81M,"Number of physical qubits for the algorithm after layout The 15,809,118 are the product of the 9,399 logical qubits after layout and the 1,682 physical qubits that encode a single logical qubit."


In [35]:
results_12_tfac_one.diagram.space

In [36]:
results_12_trapped_ion= qsharp.estimate("Main()", 
params={
                    "qubitParams": {
                        "name": "qubit_gate_us_e3"
                    }})
results_12_trapped_ion
# 12 qubits, default distillation factory, and qubit_maj_ns_e6 with floquet code error correction


Runtime,3 days,"Total runtime This is a runtime estimate for the execution time of the algorithm. In general, the execution time corresponds to the duration of one logical cycle (16,200,000 nanosecs) multiplied by the 14,427,112 logical cycles to run the algorithm. If however the duration of a single T factory (here: 163,800,000 nanosecs) is larger than the algorithm runtime, we extend the number of logical cycles artificially in order to exceed the runtime of a single T factory."
rQOPS,580.19k,"Reliable quantum operations per second The value is computed as the number of logical qubits after layout (9,399) (with a logical error rate of 2.46e-15) multiplied by the clock frequency (61.73), which is the number of logical cycles per second."
Physical qubits,13.93M,"Number of physical qubits This value represents the total number of physical qubits, which is the sum of 13,703,742 physical qubits to implement the algorithm logic, and 229,320 physical qubits to execute the T factories that are responsible to produce the T states that are consumed by the algorithm."
Logical algorithmic qubits,9.40k,"Number of logical qubits for the algorithm after layout Laying out the logical qubits in the presence of nearest-neighbor constraints requires additional logical qubits. In particular, to layout the $Q_{\rm alg} = 4,603$ logical qubits in the input algorithm, we require in total $2 \cdot Q_{\rm alg} + \lceil \sqrt{8 \cdot Q_{\rm alg}}\rceil + 1 = 9,399$ logical qubits."
Algorithmic depth,14.43M,"Number of logical cycles for the algorithm To execute the algorithm using _Parallel Synthesis Sequential Pauli Computation_ (PSSPC), operations are scheduled in terms of multi-qubit Pauli measurements, for which assume an execution time of one logical cycle. Based on the input algorithm, we require one multi-qubit measurement for the 1,562 single-qubit measurements, the 654,700 arbitrary single-qubit rotations, and the 3,275,900 T gates, three multi-qubit measurements for each of the 1,050 CCZ and 0 CCiX gates in the input program, as well as 22 multi-qubit measurements for each of the 476,900 non-Clifford layers in which there is at least one single-qubit rotation with an arbitrary angle rotation."
Logical depth,14.43M,"Number of logical cycles performed This number is usually equal to the logical depth of the algorithm, which is 14,427,112. However, in the case in which a single T factory is slower than the execution time of the algorithm, we adjust the logical cycle depth to exceed the T factory's execution time."
Clock frequency,62,Number of logical cycles per second This is the number of logical cycles that can be performed within one second. The logical cycle time is 16 millisecs.
Number of T states,17.68M,"Number of T states consumed by the algorithm To execute the algorithm, we require one T state for each of the 3,275,900 T gates, four T states for each of the 1,050 CCZ and 0 CCiX gates, as well as 22 for each of the 654,700 single-qubit rotation gates with arbitrary angle rotation."
Number of T factories,13,"Number of T factories capable of producing the demanded 17,683,500 T states during the algorithm's runtime The total number of T factories 13 that are executed in parallel is computed as $\left\lceil\dfrac{\text{T states}\cdot\text{T factory duration}}{\text{T states per T factory}\cdot\text{algorithm runtime}}\right\rceil = \left\lceil\dfrac{17,683,500 \cdot 163,800,000\;\text{ns}}{1 \cdot 233,719,214,400,000\;\text{ns}}\right\rceil$"
Number of T factory invocations,1.36M,"Number of times all T factories are invoked In order to prepare the 17,683,500 T states, the 13 copies of the T factory are repeatedly invoked 1,360,270 times."
Physical algorithmic qubits,13.70M,"Number of physical qubits for the algorithm after layout The 13,703,742 are the product of the 9,399 logical qubits after layout and the 1,458 physical qubits that encode a single logical qubit."


In [37]:
results_12_majorana= qsharp.estimate("Main()", 
params={
                    "qubitParams": {
                        "name": "qubit_maj_ns_e6"
                    },
                    "qecScheme": {
                        "name": "floquet_code"
                    }})
results_12_majorana
# 12 qubits, default distillation factory, and qubit_maj_ns_e6 with floquet code error correction


Runtime,30 secs,"Total runtime This is a runtime estimate for the execution time of the algorithm. In general, the execution time corresponds to the duration of one logical cycle (2,100 nanosecs) multiplied by the 14,427,112 logical cycles to run the algorithm. If however the duration of a single T factory (here: 21,900 nanosecs) is larger than the algorithm runtime, we extend the number of logical cycles artificially in order to exceed the runtime of a single T factory."
rQOPS,4.48G,"Reliable quantum operations per second The value is computed as the number of logical qubits after layout (9,399) (with a logical error rate of 2.46e-15) multiplied by the clock frequency (476,190.48), which is the number of logical cycles per second."
Physical qubits,2.33M,"Number of physical qubits This value represents the total number of physical qubits, which is the sum of 2,293,356 physical qubits to implement the algorithm logic, and 34,320 physical qubits to execute the T factories that are responsible to produce the T states that are consumed by the algorithm."
Logical algorithmic qubits,9.40k,"Number of logical qubits for the algorithm after layout Laying out the logical qubits in the presence of nearest-neighbor constraints requires additional logical qubits. In particular, to layout the $Q_{\rm alg} = 4,603$ logical qubits in the input algorithm, we require in total $2 \cdot Q_{\rm alg} + \lceil \sqrt{8 \cdot Q_{\rm alg}}\rceil + 1 = 9,399$ logical qubits."
Algorithmic depth,14.43M,"Number of logical cycles for the algorithm To execute the algorithm using _Parallel Synthesis Sequential Pauli Computation_ (PSSPC), operations are scheduled in terms of multi-qubit Pauli measurements, for which assume an execution time of one logical cycle. Based on the input algorithm, we require one multi-qubit measurement for the 1,562 single-qubit measurements, the 654,700 arbitrary single-qubit rotations, and the 3,275,900 T gates, three multi-qubit measurements for each of the 1,050 CCZ and 0 CCiX gates in the input program, as well as 22 multi-qubit measurements for each of the 476,900 non-Clifford layers in which there is at least one single-qubit rotation with an arbitrary angle rotation."
Logical depth,14.43M,"Number of logical cycles performed This number is usually equal to the logical depth of the algorithm, which is 14,427,112. However, in the case in which a single T factory is slower than the execution time of the algorithm, we adjust the logical cycle depth to exceed the T factory's execution time."
Clock frequency,476.19k,Number of logical cycles per second This is the number of logical cycles that can be performed within one second. The logical cycle time is 2 microsecs.
Number of T states,17.68M,"Number of T states consumed by the algorithm To execute the algorithm, we require one T state for each of the 3,275,900 T gates, four T states for each of the 1,050 CCZ and 0 CCiX gates, as well as 22 for each of the 654,700 single-qubit rotation gates with arbitrary angle rotation."
Number of T factories,13,"Number of T factories capable of producing the demanded 17,683,500 T states during the algorithm's runtime The total number of T factories 13 that are executed in parallel is computed as $\left\lceil\dfrac{\text{T states}\cdot\text{T factory duration}}{\text{T states per T factory}\cdot\text{algorithm runtime}}\right\rceil = \left\lceil\dfrac{17,683,500 \cdot 21,900\;\text{ns}}{1 \cdot 30,296,935,200\;\text{ns}}\right\rceil$"
Number of T factory invocations,1.36M,"Number of times all T factories are invoked In order to prepare the 17,683,500 T states, the 13 copies of the T factory are repeatedly invoked 1,360,270 times."
Physical algorithmic qubits,2.29M,"Number of physical qubits for the algorithm after layout The 2,293,356 are the product of the 9,399 logical qubits after layout and the 244 physical qubits that encode a single logical qubit."


In [ ]:
%%qsharp
//Grover's Search taken from https://learn.microsoft.com/en-us/azure/quantum/tutorial-qdk-grovers-search?tabs=tabid-vscode#the-grovers-algorithm
// This is for 24 qubits

import Microsoft.Quantum.Convert.*;
import Microsoft.Quantum.Math.*;
import Microsoft.Quantum.Arrays.*;
import Microsoft.Quantum.Measurement.*;
import Microsoft.Quantum.Diagnostics.*;
import Microsoft.Quantum.ResourceEstimation.*;


@EntryPoint()
operation Main24() : Result[] {
    let nQubits = 24;
    let iterations = CalculateOptimalIterations(nQubits);
    Message($"Number of iterations: {iterations}");

    // Use Grover's algorithm to find a particular marked state.
    let results = GroverSearch(nQubits, iterations, ReflectAboutMarked);
    return results;
}

operation GroverSearch(
    nQubits : Int,
    iterations : Int,
    phaseOracle : Qubit[] => Unit) : Result[] {

    use qubits = Qubit[nQubits];

    PrepareUniform(qubits);

    for _ in 1..iterations {
        // if (BeginEstimateCaching("GroverAESExample.EstimateIteration", SingleVariant())) {
            phaseOracle(qubits);
            ReflectAboutUniform(qubits);
            //EstimatingAES(qubits);
        //     EndEstimateCaching();
        // }
    }

    // Measure and return the answer.
    return MResetEachZ(qubits);
}

function CalculateOptimalIterations(nQubits : Int) : Int {
    if nQubits > 63 {
        fail "This sample supports at most 63 qubits.";
    }
    let nItems = 1 <<< nQubits; // 2^nQubits
    let angle = ArcSin(1. / Sqrt(IntAsDouble(nItems)));
    let iterations = Round(0.25 * PI() / angle - 0.5);
    return iterations;
}

operation ReflectAboutMarked(inputQubits : Qubit[]) : Unit {
    Message("Reflecting about marked state...");
    use outputQubit = Qubit();
    within {
        // We initialize the outputQubit to (|0⟩ - |1⟩) / √2, so that
        // toggling it results in a (-1) phase.
        X(outputQubit);
        H(outputQubit);
        // Flip the outputQubit for marked states.
        // Here, we get the state with alternating 0s and 1s by using the X
        // operation on every other qubit.
        for q in inputQubits[...2...] {
            X(q);
        }
        
    } apply {
        Controlled X(inputQubits, outputQubit);
        EstimatingAES(inputQubits);
    }
}

operation PrepareUniform(inputQubits : Qubit[]) : Unit is Adj + Ctl {
    for q in inputQubits {
        H(q);
    }
}

operation ReflectAboutAllOnes(inputQubits : Qubit[]) : Unit {
    Controlled Z(Most(inputQubits), Tail(inputQubits));
}

operation EstimatingAES(inputQubits : Qubit[]) : Unit {
    //Scaled values from scaling.py
    //Based on the values from Jang et al.
    AccountForEstimates(
        [TCount(91094), AuxQubitCount(5031), RotationCount(17931), RotationDepth(13231), MeasurementCount(12)],
        PSSPCLayout(), inputQubits);
}
operation ReflectAboutUniform(inputQubits : Qubit[]) : Unit {
    within {
        // Transform the uniform superposition to all-zero.
        Adjoint PrepareUniform(inputQubits);
        // Transform the all-zero state to all-ones
        for q in inputQubits {
            X(q);
        }
    } apply {
        // Now that we've transformed the uniform superposition to the
        // all-ones state, reflect about the all-ones state, then let the
        // within/apply block transform us back.
        ReflectAboutAllOnes(inputQubits);
    }


}

In [ ]:
result_24_default = qsharp.estimate("Main24()")
result_24_default['jobParams']
# 12 qubits, automatic number of distillation factories, and qubit_gate_ns_e3

{'qecScheme': {'name': 'surface_code',
  'errorCorrectionThreshold': 0.01,
  'crossingPrefactor': 0.03,
  'logicalCycleTime': '(4 * twoQubitGateTime + 2 * oneQubitMeasurementTime) * codeDistance',
  'physicalQubitsPerLogicalQubit': '2 * codeDistance * codeDistance',
  'maxCodeDistance': 50},
 'errorBudget': 0.001,
 'qubitParams': {'instructionSet': 'GateBased',
  'name': 'qubit_gate_ns_e3',
  'oneQubitMeasurementTime': '100 ns',
  'oneQubitGateTime': '50 ns',
  'twoQubitGateTime': '50 ns',
  'tGateTime': '50 ns',
  'oneQubitMeasurementErrorRate': 0.001,
  'oneQubitGateErrorRate': 0.001,
  'twoQubitGateErrorRate': 0.001,
  'tGateErrorRate': 0.001,
  'idleErrorRate': 0.001},
 'constraints': {'maxDistillationRounds': 3},
 'estimateType': 'singlePoint'}

In [46]:
result_24_default.diagram.space

In [ ]:
result_12_default = qsharp.estimate("Main24()")
result_12_default['jobParams']
# 12 qubits, automatic number of distillation factories, and qubit_gate_ns_e3

{'qecScheme': {'name': 'surface_code',
  'errorCorrectionThreshold': 0.01,
  'crossingPrefactor': 0.03,
  'logicalCycleTime': '(4 * twoQubitGateTime + 2 * oneQubitMeasurementTime) * codeDistance',
  'physicalQubitsPerLogicalQubit': '2 * codeDistance * codeDistance',
  'maxCodeDistance': 50},
 'errorBudget': 0.001,
 'qubitParams': {'instructionSet': 'GateBased',
  'name': 'qubit_gate_ns_e3',
  'oneQubitMeasurementTime': '100 ns',
  'oneQubitGateTime': '50 ns',
  'twoQubitGateTime': '50 ns',
  'tGateTime': '50 ns',
  'oneQubitMeasurementErrorRate': 0.001,
  'oneQubitGateErrorRate': 0.001,
  'twoQubitGateErrorRate': 0.001,
  'tGateErrorRate': 0.001,
  'idleErrorRate': 0.001},
 'constraints': {'maxDistillationRounds': 3},
 'estimateType': 'singlePoint'}